In [46]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error, f1_score, accuracy_score, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor

from matplotlib import pyplot as plt

0.20.2


In [84]:
def prepare_boston_data():
    data = load_boston()
    #print(data.DESCR)
    X, y = data['data'], data['target']
    #scaler = StandardScaler()
    #X = scaler.fit_transform(X)
    #X = pd.DataFrame(data=scaler.fit_transform(X), columns=data.feature_names)
    # Нормализовать даннные с помощью стандартной нормализации
    #X = (X - X.mean(axis=0)) / X.std(axis=0)
    # Добавить фиктивный столбец единиц (bias линейной модели)
    #X = np.hstack([np.ones(X.shape[0])[:, np.newaxis], X])
    return X, y

In [85]:
X, y = prepare_boston_data()

In [86]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75, shuffle=False)

In [39]:
def print_regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    print(f'MSE = {mse:.2f}, RMSE = {rmse:.2f}')

def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]

def gbm_predict(X):
    y_1 = []
    y_2 = []
    for x in X:
        s = 0
        print('[x]:', [x])
        for algo, coeff in zip(base_algorithms_list, coefficients_list):
            y_1.append(coeff * algo.predict([x])[0])
            s+= coeff * algo.predict([x])[0]
            print('coeff: ', coeff, 'algo.predict([x])[0]:', algo.predict([x])[0])
        print(y_1, s)
        y_2.append(s)
    return y_2
                
            
    #return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]

In [40]:
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]

In [41]:
base_algorithms_list = []
num_iter = 50
a = 0.9
coefficients_list = []

for i in range(0,num_iter):
    algo = DecisionTreeRegressor(max_depth=5,random_state=139)
    y_pred = y_train - gbm_predict(X_train)
    algo.fit(X_train, y_pred)
    coefficients_list.append(0.9)
    base_algorithms_list.append(algo)
    
    
y_pred = gbm_predict(X_test)
print_regression_metrics(y_test, y_pred)

MSE = 44.66, RMSE = 6.68


In [47]:
base_algorithms_list = []
num_iter = 50
a = 0.9
coefficients_list = []

for i in range(0,num_iter):
    algo = DecisionTreeRegressor(max_depth=5,random_state=139)
    y_pred = y_train - gbm_predict(X_train)
    algo.fit(X_train, y_pred)
    alpha = 0.9/(1.0+i)
    coefficients_list.append(alpha)
    base_algorithms_list.append(algo)
    
    
y_pred = gbm_predict(X_test)
print_regression_metrics(y_test, y_pred)

MSE = 43.94, RMSE = 6.63


In [87]:
class GBM_predictor():
    def __init__(self, lr=0.01, n_estimators=10, max_depth=5, random_state=139):
        self.lr = lr
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.random_state = random_state
        
    def fit(self, X, y):
        self.__last_predict = 0
        self.trees = np.full(self.n_estimators, DecisionTreeRegressor)
        self.nu_arr = np.full(self.n_estimators, self.lr)
        for i in range(self.n_estimators):
            self.nu_arr[i] = self.lr(i) if callable(self.lr) else self.lr
            self.trees[i] = DecisionTreeRegressor(max_depth=self.max_depth, random_state=self.random_state)
            
            s = y - self.__last_predict
            self.trees[i].fit(X,s)
            self.__last_predict += self.nu_arr[i] * self.trees[i].predict(X)
            
    def predict(self, X):
        trees = self.trees
        nu_arr = self.nu_arr
        return np.array([sum([coeff * algo.predict([x])[0] for algo, coeff in zip(trees, nu_arr)]) for x in X])
        

In [88]:
predictor = GBM_predictor(lr=0.9, n_estimators=50, max_depth=5)
predictor.fit(X_train, y_train)
print_regression_metrics(y_test, predictor.predict(X_test))

MSE = 44.56, RMSE = 6.68


In [89]:
from sklearn.linear_model import LinearRegression
linear = LinearRegression()
linear.fit(X_train, y_train)
print_regression_metrics(y_test, linear.predict(X_test))

MSE = 1922.48, RMSE = 43.85
